In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import chirp
import librosa
import librosa.display
import os
import random

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# changing the working directory
%cd '/content/drive/MyDrive/Experiment2'

/content/drive/MyDrive/Experiment2


In [4]:
!ls

chirp_generator.ipynb				       Synthetic_Data_Generated_MS_PCEN_for_DC
feature_extraction_training_and_valdiation_hw.ipynb    Training_Audio_Files
feature_extraction_training_and_valdiation_nohw.ipynb  Training_MS_PCEN
model.ipynb					       Validation_Audio_Files
Synthetic_Backgrounds				       Validation_MS_PCEN


In [5]:
def cr_generator():
  np.random.seed(42)
  random.seed(42)
  audio_dir = r'/content/drive/MyDrive/Experiment2/Synthetic_Backgrounds'
  audio_files = os.listdir(audio_dir)


  starting_frequencies = np.array([80, 80, 60, 100, 100, 120])
  starting_frequencies = np.tile(starting_frequencies, 30)

  chirp_rates = np.array([13, 10, 25, 40, 30, 40])
  chirp_rates = np.tile(chirp_rates, 30)

  durations_cr = np.array([3., 4., 1.2, 1., 1.35, 1.5])
  durations_cr = np.tile(durations_cr, 30)

  start_times = np.random.uniform(low=0, high=15, size=(len(audio_files),))

  indices_DC_repeated = np.asarray(random.sample(range(0, len(audio_files)), 60))
  doubling_DC = np.zeros(len(audio_files),)
  doubling_DC[indices_DC_repeated] = 1

  duration = 20
  fs = 6000
  count = 1
  count_2 = 1
  signals = []
  for i in range(len(audio_files)):
    start_time = start_times[i]
    duration_cr = durations_cr[i]
    end_time = start_time + duration_cr
    f0 = starting_frequencies[i]
    chirp_rate = chirp_rates[i]  # chirp rate in Hz/s

    print(f"The data sample {audio_files[i]} is processed: {count_2}/{len(os.listdir(audio_dir))}")
    count_2+=1
    audio_path = os.path.join(audio_dir, audio_files[i])
    signal, _ = librosa.load(audio_path, sr=fs)

    t = np.linspace(0, duration, int(fs * duration))

    if doubling_DC[i] == 1:
      print(count)
      count +=1
      # First chirp signal
      signal_cr = np.zeros_like(t)
      t_cr = np.linspace(0, end_time-start_time, int(fs * (end_time-start_time)))
      chirp_signal = chirp(t_cr,
                    f0=f0,
                    t1=end_time-start_time,
                    f1=f0 - chirp_rate * (end_time-start_time))
      start_ind = np.where(t==np.min(t[t>= start_time]))[0][0]
      end_ind = start_ind + chirp_signal.shape[0]
      signal_cr[start_ind:end_ind] = chirp_signal
      # Second chirp signal
      signal_cr_2 = np.zeros_like(t)
      if i < 180-1:
        start_time_cr_2 = start_times[i+1]
        end_time_cr_2 = start_time_cr_2 + duration_cr
        t_cr2 = np.linspace(0, end_time_cr_2-start_time_cr_2, int(fs * (end_time_cr_2-start_time_cr_2)))
        chirp_signal2 = chirp(t_cr2,
                       f0=f0,
                       t1=end_time_cr_2-start_time_cr_2,
                       f1=f0 - chirp_rate * (end_time_cr_2-start_time_cr_2))
        start_ind_2 = np.where(t==np.min(t[t>= start_time_cr_2]))[0][0]
        end_ind_2 = start_ind_2 + chirp_signal2.shape[0]
        signal_cr_2[start_ind_2:end_ind_2] = chirp_signal2
      else: # means i==180
        start_time_cr_2 = start_times[i-1]
        end_time_cr_2 = start_time_cr_2 + duration_cr
        t_cr2 = np.linspace(0, end_time_cr_2-start_time_cr_2, int(fs * (end_time_cr_2-start_time_cr_2)))
        chirp_signal2 = chirp(t_cr2,
                       f0=f0,
                       t1=end_time_cr_2-start_time_cr_2,
                       f1=f0 - chirp_rate * (end_time_cr_2-start_time_cr_2))
        start_ind_2 = np.where(t==np.min(t[t>= start_time_cr_2]))[0][0]
        end_ind_2 = start_ind_2 + chirp_signal2.shape[0]
        signal_cr_2[start_ind_2:end_ind_2] = chirp_signal2

      signal = signal + signal_cr/100 + signal_cr_2/100

    else:
      signal_cr = np.zeros_like(t)
      t_cr = np.linspace(0, end_time-start_time, int(fs * (end_time-start_time)))
      chirp_signal = chirp(t_cr,
                    f0=f0,
                    t1=end_time-start_time,
                    f1=f0 - chirp_rate * (end_time-start_time))
      start_ind = np.where(t==np.min(t[t>= start_time]))[0][0]
      end_ind = start_ind + chirp_signal.shape[0]
      signal_cr[start_ind : end_ind] = chirp_signal
      signal = signal + signal_cr/100

    signals.append(signal)
  return signals

In [6]:
signals = cr_generator()

The data sample 52CDE419_0.wav is processed: 1/180
The data sample 52A41D90_190.wav is processed: 2/180
1
The data sample 52A1068E_100.wav is processed: 3/180
The data sample 530E17A5_60.wav is processed: 4/180
The data sample 530EB5D9_50.wav is processed: 5/180
The data sample 52A2386A_10.wav is processed: 6/180
The data sample 52A41D90_100.wav is processed: 7/180
2
The data sample 53E94889_100.wav is processed: 8/180
3
The data sample 53E94889_50.wav is processed: 9/180
4
The data sample 53E94889_0.wav is processed: 10/180
The data sample 5350AEEA_0.wav is processed: 11/180
5
The data sample 53E94889_160.wav is processed: 12/180
6
The data sample 5350AEEA_120.wav is processed: 13/180
The data sample 5350AEEA_200.wav is processed: 14/180
The data sample 542F1D7D_0.wav is processed: 15/180
The data sample 5416817D_280.wav is processed: 16/180
The data sample 540A13D9_160.wav is processed: 17/180
The data sample 542EB081_200.wav is processed: 18/180
7
The data sample 54336D64_200.wav is

In [7]:
signals = np.array(signals)

In [8]:
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

spectrogram_dir = r'/content/drive/MyDrive/Experiment2/Synthetic_Data_Generated_MS_PCEN_for_DC'

Training = True # Statement: If it is true, you will generate MS-PCEN with agumentation

os.makedirs(spectrogram_dir, exist_ok=True)

count = 0

for i in range(signals.shape[0]):
    audio = signals[i]
    # Mel
    spectrogram = librosa.feature.melspectrogram(y=audio, sr=6000, n_fft=2048,
                                                 hop_length=512, win_length=2048,
                                                 n_mels=128, power=1) # I think he was using 128 as default
    mel_pcen = librosa.pcen(spectrogram, sr=6000, hop_length=512,
                            gain=0.98, bias=0.2, power=0.9,
                            time_constant=0.0004, eps=1e-6)
    plt.figure(figsize=(8, 4))
    librosa.display.specshow(mel_pcen, sr=6000, x_axis='time', y_axis='mel')
    plt.tight_layout()
    save_path = os.path.join(spectrogram_dir, f'{i}synthetic.png')
    plt.axis('off')
    plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
    plt.close()
    # Following 6 lines are to convert CYMK images to RGB images with the grayscale values ranging between 0-255
    spec_image = Image.open(save_path)
    spec_image_rgb = spec_image.convert('RGB')
    rgb_array = np.array(spec_image_rgb)
    rgb_array_norm = 255*((rgb_array-np.min(rgb_array))/(np.max(rgb_array)-np.min(rgb_array)))
    rgb_image_spec = Image.fromarray(rgb_array_norm.astype(np.uint8))
    rgb_image_spec.save(save_path)